In [3]:
# !pip install pinecone-client
# !pip install -U sentence-transformers

In [1]:
from pinecone import Pinecone, ServerlessSpec
import os

pinecone_api = os.environ["PINECONE_API_KEY"]

# vectorstore load
pc = Pinecone(api_key=pinecone_api)
spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

index_name = "ragtest"

# if not have that index
pc.create_index(
    name=index_name,
    dimension=1536,
    metric="cosine",
    spec=spec,
)

c:\Users\USER\miniconda3\envs\20240909\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
index = pc.Index(index_name)  # 인덱스 연결 

In [6]:
# !pip install langchain-pinecone

  Using cached langchain_core-0.3.33-py3-none-any.whl.metadata (6.3 kB)
Using cached langchain_core-0.3.33-py3-none-any.whl (412 kB)
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 1.1.0
    Uninstalling pinecone-plugin-inference-1.1.0:
      Successfully uninstalled pinecone-plugin-inference-1.1.0
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone-client 5.0.1 requires pinecone-plugin-inference<2.0.0,>=1.0.3, but you have pinecone-plugin-inference 3.1.0 which is incompatible.


In [7]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [9]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(embedding=embeddings, index=index)
vector_store.add_documents(documents=page_all)

NameError: name 'page_all' is not defined

In [ ]:
import os
import time
import json
from pinecone import Pinecone, ServerlessSpec
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
from getpass import getpass

# 환경 변수 로드
load_dotenv()

# API 키 입력받기 (환경 변수에 없으면 수동 입력)
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter API key for OpenAI: ")

if not os.environ.get("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass("Enter Pinecone API key: ")

pinecone_api = os.environ["PINECONE_API_KEY"]
openai_api_key = os.environ["OPENAI_API_KEY"]

# Pinecone 설정
pc = Pinecone(api_key=pinecone_api)
spec = ServerlessSpec(cloud="aws", region="us-east-1")

index_name = "ragtest"

# 인덱스 생성 (이미 존재하면 건너뜀)
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=1536,  # 'dimention' 오타 수정
        metric="cosine",
        spec=spec,
    )
    print("✅ Pinecone 인덱스 생성 완료!")

# 인덱스 준비 확인 (서버리스에서는 불필요하지만 포함)
while not pc.describe_index(index_name).status["ready"]:
    time.sleep(1)

index = pc.Index(index_name)

# 🔹 OpenAI Embeddings 설정
embed_model = OpenAIEmbeddings(model="text-embedding-3-large", openai_api_key=openai_api_key)

# 🔹 JSON 데이터 로드
with open("data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 🔹 텍스트를 벡터로 변환
vectors = []
for item in data:
    embedding = embed_model.embed_query(item["text"])  # OpenAI Embedding 사용
    vectors.append({
        "id": item["id"],
        "values": embedding,
        "metadata": {'text': item["text"]}
    })

# 🔹 Pinecone에 데이터 업로드
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

# 🔹 인덱스 상태 확인
print("✅ 데이터 업로드 완료!")
print(index.describe_index_stats())


In [ ]:
import os
import json
from glob import glob  # 여러 JSON 파일 처리
from pinecone import Pinecone, ServerlessSpec
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
from getpass import getpass

# 환경 변수 로드
load_dotenv()

# API 키 설정 (없으면 직접 입력받기)
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter API key for OpenAI: ")

if not os.environ.get("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass("Enter Pinecone API key: ")

pinecone_api = os.environ["PINECONE_API_KEY"]
openai_api_key = os.environ["OPENAI_API_KEY"]

# 🔹 Pinecone 연결
pc = Pinecone(api_key=pinecone_api)
spec = ServerlessSpec(cloud="aws", region="us-east-1")

index_name = "camera_models"  # 단일 인덱스

# 인덱스 생성 (이미 존재하면 스킵)
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=1536,  # OpenAI embedding 크기
        metric="cosine",
        spec=spec,
    )
    print("✅ Pinecone 인덱스 생성 완료!")

# 인덱스 준비 확인
index = pc.Index(index_name)

# 🔹 OpenAI Embeddings 설정
embed_model = OpenAIEmbeddings(model="text-embedding-3-large", openai_api_key=openai_api_key)

# 🔹 JSON 파일 불러오기
json_files = glob("data/*.json")  # 'data' 폴더 내 모든 JSON 파일 가져오기

print(f"📂 {len(json_files)}개의 카메라 모델 JSON 파일을 찾았습니다.")

# 🔹 JSON 파일별로 네임스페이스를 지정하여 데이터 저장
for file_path in json_files:
    model_name = os.path.splitext(os.path.basename(file_path))[0]  # 파일명 -> 카메라 모델명

    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    vectors = []
    for item in data:
        embedding = embed_model.embed_query(item["text"])
        vectors.append({
            "id": item["id"],  # 고유 ID
            "values": embedding,  # 벡터 값
            "metadata": {'text': item["text"]}  # 원본 텍스트 저장
        })

    # 🔹 Pinecone에 네임스페이스별로 업로드
    index.upsert(vectors=vectors, namespace=model_name)
    print(f"📤 {model_name} 데이터 업로드 완료!")

print("✅ 모든 카메라 모델 데이터 업로드 완료!")


In [ ]:
# import os
# from pinecone import Pinecone, ServerlessSpec
# from dotenv import load_dotenv


# load_dotenv()

# if not os.environ.get("OPENAI_API_KEY"):
#   os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

# if not os.environ.get("PINECONE_API_KEY"):
#   os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter Pinecone API key: ")

# pinecone_api = os.environ["PINECONE_API_KEY"]

# # vectorstore load
# pc = Pinecone(api_key=pinecone_api)
# spec = ServerlessSpec(
#     cloud="aws", region="us-east-1"
# )

# index_name = "ragtest"

# # if not have that index
# pc.create_index(
#     name=index_name,
#     dimension=1536,
#     metric="cosine",
#     spec=spec,
# )

# # Wait for the index to be ready
# while not pc.describe_index(index_name).status['ready']:
#     time.sleep(1)

# index = pc.Index(index_name)


# from langchain.embeddings.openai import OpenAIEmbeddings

# embed_model = OpenAIEmbeddings(model="text-embedding-3-large")

# vectors = []
# for d, e in zip(data, embeddings):
#     vectors.append({
#         "id": d['id'],
#         "values": e['values'],
#         "metadata": {'text': d['text']}
#     })

# index.upsert(
#     vectors=vectors,
#     namespace="ns1"
# )

# print(index.describe_index_stats())

In [ ]:
from pinecone import Pinecone

# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [2]:
from dotenv import load_dotenv
import os

load_dotenv(override=True) # 강제 다시 로드

pinecone_api = os.environ["PINECONE_API_KEY"]

